# ECMWF Tropical Cyclone Data Extractor - LATEST STORM Example

The extractor downloads tropical cyclone track BUFR files from ECMWF's Dissemination (DISS) system at https://essential.ecmwf.int/. It provides dynamic access to forecast data based on date/time parameters.

## Download

In [ ]:
from ecmwf_tc_data_downloader import download_tc_data, list_available_storms

import os
import shutil
import glob

### List available storms

In [ ]:
storms = list_available_storms(limit=1)

print(f"Found {len(storms)} unique storms:")
for i, storm in enumerate(storms, 1):
    print(f"{i:2d}. {storm['storm_name']:8s} at {storm['latitude']:12s}, {storm['longitude']:12s} (Forecast: {storm['forecast_time']})")

### Download latest forecasts

In [ ]:
# Clear the tc_latest directory before processing
if os.path.exists('tc_latest'):
    shutil.rmtree('tc_latest')
    print("Cleared tc_latest directory")

# Ensure tc_latest directory exists
os.makedirs('tc_latest', exist_ok=True)

In [ ]:
results = download_tc_data(limit=1, output_dir='tc_latest')

## Extract

In [ ]:
from ecmwf_tc_data_extractor import extract_tc_data, extract_tc_data_from_file

In [ ]:
# Get files from tc_latest directory, preferring named storms over numeric identifiers
bin_files = glob.glob('tc_latest/*.bin')
if not bin_files:
    raise FileNotFoundError("No .bin files found in tc_latest directory")

# Prefer files with named storms (not just numeric identifiers like 24W, 23W)
# Look for files that have a name after "tropical_cyclone_track_" that doesn't start with a number
named_storm_files = []
numeric_storm_files = []

for file in bin_files:
    # Extract the part after "tropical_cyclone_track_"
    if 'tropical_cyclone_track_' in file:
        after_track = file.split('tropical_cyclone_track_')[1]
        storm_name = after_track.split('_')[0]  # Get the first part before the next underscore

        # Check if it's a named storm (doesn't start with a number)
        if not storm_name[0].isdigit():
            named_storm_files.append(file)
        else:
            numeric_storm_files.append(file)

# Prefer named storms, fall back to numeric if no named storms available
if named_storm_files:
    filename = named_storm_files[0]
    storm_name = filename.split('tropical_cyclone_track_')[1].split('_')[0]
    print(f"Selected named storm file: {storm_name} - {filename}")
else:
    filename = numeric_storm_files[0] if numeric_storm_files else bin_files[0]
    storm_name = filename.split('tropical_cyclone_track_')[1].split('_')[0] if 'tropical_cyclone_track_' in filename else "unknown"
    print(f"No named storms found, using: {storm_name} - {filename}")


In [ ]:
# Extract using the main function (saves to CSV automatically)
result = extract_tc_data_from_file(filename, output_dir='tc_latest')

### Extract data without saving to CSV

In [ ]:
df = extract_tc_data(filename, verbose=False)

print(f"Extracted {len(df)} records")
print(f"Columns: {list(df.columns)}")
print(f"Sample data:")
print(df.head())

## Explore

In [ ]:
from ecmwf_tc_data_explorer import (
    create_track_visualization,
    create_intensity_plot,
    create_wind_field_visualization
)

In [ ]:
# Load the extracted data - find the CSV file automatically
csv_files = glob.glob('tc_latest/*_extracted.csv')
if not csv_files:
    raise FileNotFoundError("No extracted CSV files found in tc_latest directory")

csv_file = csv_files[0]
print(f"Loading data from: {csv_file}")

In [ ]:
fig_tracks = create_track_visualization(csv_file)
fig_tracks

In [ ]:
fig_wind = create_wind_field_visualization(csv_file)

In [ ]:
fig_intensity = create_intensity_plot(csv_file)